In [1]:
import cv2

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
net=cv2.dnn.readNetFromDarknet("yolo.cfg","yolov3-spp.weights")

In [5]:
classes=[]
with open('coco.names','r') as f:
    classes=[line.strip() for line in f.readlines()]

In [ ]:
vid = cv2.VideoCapture(0)
from collections import defaultdict
#"Man Placing a Book and his Feet on a Table - Free Stock Creative Commons Video.mp4"
ht,wt,ch=480,640,3
last_layer=net.getUnconnectedOutLayersNames()
objects1={}
objects={}
while(vid.isOpened()):
    ret, frame = vid.read()
    track= defaultdict(lambda: 0)
    frame=cv2.resize(frame,(640,480))
    try:
        blob=cv2.dnn.blobFromImage(frame,1/255,(416,416),(0,0,0),swapRB=True,crop=False)
    except:
        pass
    net.setInput(blob)
    layer_out=net.forward(last_layer)
    boxes=[]
    confidences=[]
    classids=[]
    for output in layer_out:
        for detection in output:
            score=detection[5:]
            classid=np.argmax(score)
            confidence=score[classid]
            if confidence > .6:
                center_x=int(detection[0] * wt)
                center_y=int(detection[1] * ht)
                w=int(detection[2] * wt)
                h=int(detection[3] * ht)
                boxes.append([center_x,center_y,w,h])
                confidences.append((float(confidence)))
                classids.append(classid)
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.4)
    indexes=np.unique(indexes,axis=0)
    font=cv2.FONT_HERSHEY_SIMPLEX
    colors=np.random.uniform(0,255,size=(len(boxes),3)) 

    try:
        for i in indexes.flatten():
            x,y,w,h=boxes[i]
            label=str(classes[classids[i]])
            track[label]+=1
            confidence=str(round(confidences[i],2))
            color=colors[i]
            left=round(x-(w/2))
            top=round(y-(h/2))
            right=round(x+(w/2))
            bottom=round(y+(h/2))
            objects1[label+str(track[label])]=[[left,top],[right,bottom]]
            cv2.rectangle(frame,(left,top),(right,bottom),color,2)
            cv2.putText(frame,label+" "+str(track[label])+" "+str(confidence),(round(left+10),top-10),font,fontScale=0.5,color=(0,0,0))
            pos=((left+right)//2,(top+bottom)//2)
            objects[label+str(track[label])]=pos
            pos1=(pos[0]+5,pos[1]+10)
            cv2.putText(frame,"({},{})".format(pos[0],pos[1]),pos1,fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=0.8,color=(0,0,1))
            cv2.circle(frame,pos,3,color=(0,0,255))
    except:
        pass
    cv2.imshow('Frame',frame)
    if("Karan" in takeCommand()):
        assistant()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        vid.release()
        cv2.destroyAllWindows()

Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....
Listening....
Recognizing....


In [ ]:
objects

In [6]:
def line_distance(a,b,c,x,y):
    return a*x+b*y+c/(np.sqrt(a**2+b**2))

In [7]:
def point_distance(x1,y1,x2,y2):
    return np.sqrt((x2-x1)**2+(y2-y1)**2)

In [8]:
def direct(point1,point2,obj,height,width):
    if point1[0]>point2[0]+(width/2) and (point1[1]<=point2[1]+(height/2) and point1[1]>=point2[1]-(height/2)):
        return("It is to the right of {}".format(obj))
    elif point1[0]<point2[0]-(width/2) and (point1[1]<=point2[1]+(height/2) and point1[1]>=point2[1]-(height/2)):
        return("It is to the left of {q}".format(obj))
    elif point1[1]<point2[1]+(height/2):
        
        return("It is on the {}".format(obj))
    elif point1[1]>point2[1]-(height/2):
        return("It is under the {}".format(obj))
    else:
        return("It is with the {}".format(obj))

In [9]:
def locate(obj):
    if obj in objects.keys():
        point=objects[obj]
        l1=[0,0]
        l2=np.polyfit((640,0),(640,480),1)
        l3=np.polyfit((640,480),(480,0),1)
        l4=[0,0]
        distances={}
        l1_d=line_distance(l1[0],1,l1[1],point[0],point[1])
        l2_d=line_distance(l2[0],1,l2[1],point[0],point[1])
        l3_d=line_distance(l3[0],1,l3[1],point[0],point[1])
        l4_d=line_distance(l4[0],1,l4[1],point[0],point[1])
        distances[l1_d]='l1'
        distances[l2_d]='l2'
        distances[l3_d]='l3'
        distances[l4_d]='l4'
        for k,v in objects.items():
            if k==obj:
                continue
            else:
                coors=objects1[k]
                distances[point_distance(v[0],v[1],point[0],point[1])/((coors[1][1]-coors[0][1])*(coors[1][0]-coors[0][0]))]=k
                index=min([k for k in distances.keys()])
            #print(distances)
            min_obj=(distances[index])
        try:
            if min_obj in ['l1','l2','l3','l4']:
                return("cannot find object")
            else:
                coors=objects1[min_obj]
                height=(coors[1][1]-coors[0][1])
                width=(coors[1][0]-coors[0][0])
                return direct(point,objects[min_obj],min_obj,height,width)
        except:
            return("object is near your surroundings please look carefully!")
    else:
        return("Cannot locate object")

In [10]:
print(locate('person1'))

NameError: name 'objects' is not defined

In [11]:
### voice assistant (karen)

In [10]:
import win32com.client as wincl
s = wincl.Dispatch("SAPI.SpVoice")
import datetime
import speech_recognition as sr
import wikipedia
import webbrowser
import os
import numpy as np
import pyaudio

In [11]:
def speak(audio):
    s.Speak(audio)

In [12]:
def wish(query):
    hour=int(datetime.datetime.now().hour)
    if(hour>=0 and hour <12):
        speak("good morning sir!")
    elif(hour>=12 and hour<18):
        speak("good afternoon sir!")
    else:
        speak("good evening sir!")
    speak("i am Karan ! how may i help you?")
    query=takeCommand().lower()
    if "say hi" in query:
        speak("hii ! ellavunnaru!")

In [13]:
def takeCommand():
    r=sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening....")
        audio=r.listen(source)
    try:
        print("Recognizing....")
        query=r.recognize_google(audio,language='en-in')
    except Exception as e:
        #speak("Say that again please....")
        return "None"
    return query

In [14]:
def assistant():
    query=takeCommand().lower()
    if "hai" or "hey" or "hello" in query:
        wish(query)
    while True:
        query=takeCommand().lower()
        if ('what'and 'is' and 'the' and 'time') in query:
            hour=(datetime.datetime.now().hour)
            minute=(datetime.datetime.now().minute)
            if(hour<=12):
                speak("the time is"+str(hour)+"!"+str(minute)+"AM")
            else:
                speak("the time is"+str(hour-12)+"!"+str(minute)+"PM")
        elif 'locate' in query:
            query=query.split()
            obj=query[query.index('locate')+1]
            speak(locate(obj)+str("!sir"))
        elif 'open' in query:
            if 'youtube' in query:
                webbrowser.open("youtube.com")
            elif 'google' in query:
                webbrowser.open("google.com")
            elif 'facebook' in query:
                webbrowser.open("facebook.com")
            elif 'mail' in query:
                webbrowser.open("gmail.com")
        elif ((('who'and 'made'and 'you')in query) or (('where'and 'are'and 'you'and 'born')in query) or (('who'and 'are'and 'you')in query) or (('what'and'are'and'you')in query) ):
            speak("i am a voice assistant , i am made by my master Dr.Y phani !rathan on experimenting in his lab")
        elif 'play ' and 'music' in query:
            songs=os.listdir(r"C:\Users\User\Music")
            num=np.random.randint(2,4)
            os.startfile(os.path.join('C:/Users/User/Music',songs[num]))
        elif (('what' and 'is')in query)or (('say'and'something'and'about')in query):
            speak('please wait sir i am searching for your request')
            if 'what is' in query:
                query=query.replace("what is","")
            else:
                query=query.replace("say something about","")
            results=wikipedia.summary(query,sentences=2)
            speak("On my search i have found that !")
            speak(results)
            
        elif 'stop' in query:
            speak("Thank You sir")
            break